In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd

In [2]:
cred = credentials.Certificate("service-account.json")
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
mapping = {
    'user':'users_dev',
    'booking':'bookings_dev',
    'emailReminders':'email_reminders_dev',
    'smsReminders':'sms_reminders_dev'
}

In [ ]:
for key in mapping:
    print(mapping[key])

In [4]:
for key in mapping:
    temp = list(db.collection('{}'.format(key)).stream())
    temp_dict = list(map(lambda x: x.to_dict(), temp))
    globals()[f"df_{mapping[key]}"] = pd.DataFrame(temp_dict)

In [10]:
#df_bookings_dev.head()

In [7]:
key_path = r"C:\Users\nites\OneDrive\Documents\napo-nitesh-local-ae32-vpcservice-datawarehouse-879a160a28ee.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

job_config2 = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
)

In [8]:
dataset_name='firestore'

for key in mapping:
    load_job = client.load_table_from_dataframe(
        globals()[f"df_{mapping[key]}"]
        ,"ae32-vpcservice-datawarehouse.{}.{}".format(dataset_name,mapping[key])
        ,job_config=job_config2
        ,location='EU'
    ) 
    print(load_job.result())

c:\Users\nites\anaconda3\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:405: UserWarning: Pyarrow could not determine the type of columns: courses.
  warnings.warn(


c:\Users\nites\anaconda3\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:405: UserWarning: Pyarrow could not determine the type of columns: message.
  warnings.warn(
